create separate models for title prediction and/or text prediction and divide the dataset into title datasets and text datasets. for that need to redo clean_csv

Importing primary dependencies for data fetch and cleanup

In [ ]:
import pandas as pd
import numpy as np
import re

from pandas import read_csv

Various constants and data fetch

In [ ]:
max_features=7200
max_len=1024
test_prop=0.3
epoch_count=20
batch_size=512

In [ ]:
fake=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/fake.csv')
real=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/set1/true.csv')
data2=read_csv('https://raw.githubusercontent.com/MobiusYeeitus/FakeNewsDetection/master/datasets/cleaned/ds2.csv')

In [ ]:
fake.tail(4)

,title,text,subject,date
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016"


In [ ]:
real.tail(4)

,title,text,subject,date
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017"


Compressed every cleanup job into the *clean_csv* function

In [ ]:
def clean_string(string): #function to clean a string | to be used in clean_csv
    string=string.lower() #convert string to lowercase
    string=re.sub('\n|\r|\t','',string) #cleaning of whitespaces
    string=re.sub(r'[^\w\s]+','',string) #cleaning of symbols
    return string
def clean_csv(df,identity): #cleaning the CSV file
    print('Cleaning CSV.')
    df=df.dropna() #drop all tuples with empty attributes
    length=[]
    [length.append(len(str(text))) for text in df["text"]] #calculate length of an article
    df['length'] = length #saves the length of the artivle in a new column
    df = df.drop(df['text'][df['length'] < 120].index, axis = 0) #drop all tuples with less than 120 letters
    df['text']=df['title']+" "+df['text'] #append text to title
    vfunc=np.vectorize(clean_string) #vectorize th clean_string function for use in pandas df
    df['text']=vfunc(df['text']) #apply the vectorized function to further clean the data
    df['identity']=identity #add the identity (T/F) or (0/1) in a new attribute to the data
    df.reset_index() #reset all indexes of the dataset
    print('Cleaning completed.')
    return df

In [ ]:
fake1=fake.iloc[:int(len(fake)/2),:]
fake2=fake.iloc[int(len(fake)/2):,:]

In [ ]:
fake1=clean_csv(fake1,0)

Cleaning CSV.
Cleaning completed.


In [ ]:
fake2=clean_csv(fake2,0)

Cleaning CSV.
Cleaning completed.


In [ ]:
fake=pd.concat([fake1,fake2])

In [ ]:
fake.sample(4)

,title,text,subject,date,length,identity
13822,UNREAL! OBAMA SAYS HIROSHIMA TOUR to Honor “Al...,unreal obama says hiroshima tour to honor all ...,politics,"May 26, 2016",199,0
9031,"Jeb! Makes Awkward ‘Hurricane Katrina’ Joke, ...",jeb makes awkward hurricane katrina joke remi...,News,"January 1, 2016",2805,0
23232,Stuck on Stupid: Why is America Starting WW3?,stuck on stupid why is america starting ww3 sa...,Middle-east,"October 11, 2016",7793,0
10219,WATCH TRUMP SUPPORTERS CRASH Pro-Sanctuary Cit...,watch trump supporters crash prosanctuary city...,politics,"Aug 6, 2017",2161,0


In [ ]:
real1=real.iloc[:int(len(real)/2),:]
real2=real.iloc[int(len(real)/2):,:]

In [ ]:
real1=clean_csv(real1,1)

Cleaning CSV.
Cleaning completed.


In [ ]:
real2=clean_csv(real2,1)

Cleaning CSV.
Cleaning completed.


In [ ]:
real=pd.concat([real1,real2])

In [ ]:
real.tail(4)

,text,identity
21413,lexisnexis withdrew two products from chinese ...,1
21414,minsk cultural hub becomes haven from authorit...,1
21415,vatican upbeat on possibility of pope francis ...,1
21416,indonesia to buy 114 billion worth of russian ...,1


Dropping all unnecessary columns

In [ ]:
fake=fake[['text','identity']]
real=real[['text','identity']]

In [ ]:
fake.sample(2)

,text,identity
4215,breaking michelle obama hammers trumps behavi...,0
19031,lol chuckie schumer warns trump is in trouble ...,0


In [ ]:
data1=pd.concat([real,fake],ignore_index=True)

In [ ]:
data1.tail(4)

,text,identity
43748,justice yahoo settles email privacy classactio...,0
43749,sunnistan us and allied safe zone plan to take...,0
43750,how to blow 700 million al jazeera america fin...,0
43751,10 us navy sailors held by iranian military s...,0


In [ ]:
data=pd.concat([data1,data2],ignore_index=True)

In [ ]:
data.tail(4)

,text,identity,Unnamed: 0
61810,nfl playoffs schedule matchups and odds the n...,0,18058.0
61811,macys is said to receive takeover approach by ...,0,18059.0
61812,nato russia to hold parallel exercises in balk...,1,18060.0
61813,what keeps the f35 alive david swanson is an...,1,18061.0


In [ ]:
data=data[['text','identity']]
data.reset_index()
data.tail()

,text,identity
61809,rapper ti trump a poster child for white supre...,0
61810,nfl playoffs schedule matchups and odds the n...,0
61811,macys is said to receive takeover approach by ...,0
61812,nato russia to hold parallel exercises in balk...,1
61813,what keeps the f35 alive david swanson is an...,1


Importing dependencies for ANN and Representation

In [ ]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Conv1D,MaxPool1D,Dropout,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

Tokenizing the dataset for use in the ANN

In [ ]:
print('Tokenizing DF.')
tokenizer=Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split =' ') #filters is redundant, but still keeping if I missed any symbols previously
tokenizer.fit_on_texts(texts=data['text']) #fit
X=tokenizer.texts_to_sequences(texts=data['text'])
X=pad_sequences(sequences=X,maxlen=max_len,padding='pre') #padding the set of words
Y=data['identity'].values #identities (0/1)
V=tokenizer.word_index #vocabulary
print('Tokenized DF.')

Tokenizing DF.
Tokenized DF.


Creating the model and compiling it to be used.

In [ ]:
data

,text,identity
0,as us budget fight looms republicans flip thei...,1
1,us military to accept transgender recruits on ...,1
2,senior us republican senator let mr mueller do...,1
3,fbi russia probe helped by australian diplomat...,1
4,trump wants postal service to charge much more...,1
...,...,...
61809,rapper ti trump a poster child for white supre...,0
61810,nfl playoffs schedule matchups and odds the n...,0
61811,macys is said to receive takeover approach by ...,0
61812,nato russia to hold parallel exercises in balk...,1


In [ ]:
X

array([[   0,    0,    0, ...,    8,    1, 1052],
       [   0,    0,    0, ...,   40, 1938, 1898],
       [   0,    0,    0, ...,   12,   11,   15],
       ...,
       [   0,    0,    0, ...,    1, 2808,  935],
       [   0,    0,    0, ...,    8,   44, 4019],
       [   0,    0,    0, ...,    4,   19,  164]], dtype=int32)

In [ ]:
def create_lstm_model(max_features): #creating the internal structure of the simple LSTM model
        print('Creating LSTM model.')
        lstm_model=Sequential(name='Sierra')
        lstm_model.add(layer=Embedding(input_dim=max_features,output_dim=128,name='Echo'))
        lstm_model.add(layer=LSTM(units=128,name='Lima'))
        lstm_model.add(layer=Dense(1,activation='sigmoid',name ='Omega'))
        print('Model creation completed.')
        lstm_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
        print('Model compiled.')
        return lstm_model

In [ ]:
lstm_model=create_lstm_model(max_features)

Creating LSTM model.
Model creation completed.
Model compiled.


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y)

In [ ]:
lstm_model.fit(X_train,Y_train,batch_size=batch_size,epochs=epoch_count,verbose=1,validation_split=test_prop,shuffle=1,use_multiprocessing=True)

Epoch 1/20
64/64 [==============================] - 20s 287ms/step - loss: 0.4446 - accuracy: 0.7950 - val_loss: 0.2623 - val_accuracy: 0.8919
Epoch 2/20
64/64 [==============================] - 21s 326ms/step - loss: 0.2829 - accuracy: 0.8847 - val_loss: 0.2270 - val_accuracy: 0.9162
Epoch 3/20
64/64 [==============================] - 17s 274ms/step - loss: 0.3648 - accuracy: 0.8225 - val_loss: 0.3551 - val_accuracy: 0.8363
Epoch 4/20
64/64 [==============================] - 16s 248ms/step - loss: 0.2155 - accuracy: 0.9176 - val_loss: 0.2283 - val_accuracy: 0.9160
Epoch 5/20
64/64 [==============================] - 14s 225ms/step - loss: 0.1581 - accuracy: 0.9468 - val_loss: 0.6573 - val_accuracy: 0.7814
Epoch 6/20
64/64 [==============================] - 14s 225ms/step - loss: 0.2142 - accuracy: 0.9269 - val_loss: 0.2203 - val_accuracy: 0.9258
Epoch 7/20
64/64 [==============================] - 15s 229ms/step - loss: 0.1368 - accuracy: 0.9553 - val_loss: 0.1957 - val_accuracy: 0.9378

In [ ]:
Y_pred=(lstm_model.predict(X_test)>=0.5).astype(int)

483/483 [==============================] - 7s 13ms/step


In [ ]:
Y_pred


array([[1],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [ ]:
Y_test

array([1, 0, 1, ..., 0, 1, 0])

In [ ]:
accuracy_score(Y_test,Y_pred)

0.9306975540313187

In [ ]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      8158
           1       0.93      0.92      0.93      7296

    accuracy                           0.93     15454
   macro avg       0.93      0.93      0.93     15454
weighted avg       0.93      0.93      0.93     15454



In [ ]:
string_predict1=pd.DataFrame()
string_predict1=data.iloc[22000:23000]
string_predict1

,text,identity
22000,trumps dc hotel raised price and made million...,0
22001,trumps rhetoric just resulted in a trillion d...,0
22002,this old tweet from trump reveals the real re...,0
22003,trump is raising your healthcare premiums and...,0
22004,donald trump just announced another position ...,0
...,...,...
22995,watch teacher whitesplains his use of ner to ...,0
22996,trump accidentally says he wants single payer...,0
22997,trump nominee brags that taking away healthca...,0
22998,trump says hes going to save the country mone...,0


In [ ]:
X=tokenizer.texts_to_sequences(texts=string_predict1['text'])
X=pad_sequences(sequences=X,maxlen=max_len,padding='pre')
string_predict1_res=(lstm_model.predict(X)>=0.5).astype(int)

32/32 [==============================] - 1s 18ms/step


In [ ]:
#string_predict1_res

In [ ]:
accuracy_score(string_predict1_res,string_predict1['identity'])

1.0

In [ ]:
string_predict2=pd.DataFrame()
string_predict2=data.iloc[0:1000]
string_predict2


,text,identity
0,as us budget fight looms republicans flip thei...,1
1,us military to accept transgender recruits on ...,1
2,senior us republican senator let mr mueller do...,1
3,fbi russia probe helped by australian diplomat...,1
4,trump wants postal service to charge much more...,1
...,...,...
995,house panels seek documents on puerto rico uti...,1
996,trump releases some jfk files blocks others un...,1
997,tillerson tells myanmar army chief us concerne...,1
998,exclusive while advising trump in 2016 excia c...,1


In [ ]:
#tokenizer.fit_on_texts(texts=string_predict2['text'])
X=tokenizer.texts_to_sequences(texts=string_predict2['text'])
X=pad_sequences(sequences=X,maxlen=max_len,padding='pre')
string_predict2_res=(lstm_model.predict(X)>=0.5).astype(int)

4/4 [==============================] - 0s 28ms/step


In [ ]:
#X


In [ ]:
#string_predict2_res

In [ ]:
accuracy_score(string_predict2_res,string_predict2['identity'])

0.97

In [ ]:
#string_predict2['identity'].to_array

In [ ]:
lstm_model.save('models/ds1-2_combined_epoch-100')

In [ ]:
nmod=tf.keras.saving.load_model('models/ds1-2_combined_epoch-100')

In [ ]:
y_pred = (nmod.predict(X_test) >=0.5).astype(int)

In [ ]:
accuracy_score(Y_test,y_pred)